In [20]:
import pandas as pd
import numpy as np
df_mine=pd.read_csv('Mine.csv')
df_sales=pd.read_csv('Sales.csv')

In [21]:
print("Missing values before handling:\n",df_mine.isnull().sum())
for col in ['Duration', 'Pulse', 'Maxpulse', 'Calories']:
     if df_mine[col].dtype in ['int64', 'float64']:
      df_mine[col]=df_mine[col].fillna(df_mine[col].mean())
     else: 
       df_mine[col]=df_mine[col].fillna(df_mine[col].mode()[0])
print("checking for Missing values after handling:\n",df_mine.isnull().sum())       

Missing values before handling:
 Duration    0
Date        2
Pulse       1
Maxpulse    2
Calories    4
dtype: int64
checking for Missing values after handling:
 Duration    0
Date        2
Pulse       0
Maxpulse    0
Calories    0
dtype: int64


In [22]:
print("Missing values before handling:\n",df_sales.isnull().sum())
for col in ['Order ID','Customer Name','Order Date','Product','Quantity','Unit Price','Total Revenue']:
    if df_sales[col].dtype in ['int64', 'float64']:
      df_sales[col].fillna(df_sales[col].mean(), inplace=True)
    else: 
       df_sales[col].fillna(df_sales[col].mode()[0], inplace=True)
print("checking for Missing values after handling:\n",df_sales.isnull().sum())       

Missing values before handling:
 Order ID         0
Customer Name    1
Order Date       0
Product          0
Quantity         1
Unit Price       1
Total Revenue    1
dtype: int64
checking for Missing values after handling:
 Order ID         0
Customer Name    0
Order Date       0
Product          0
Quantity         0
Unit Price       0
Total Revenue    0
dtype: int64


C:\Users\MAKOS\AppData\Local\Temp\ipykernel_17108\2348644191.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_sales[col].fillna(df_sales[col].mean(), inplace=True)
C:\Users\MAKOS\AppData\Local\Temp\ipykernel_17108\2348644191.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.


In [23]:
print(df_mine.dtypes)
print(df_sales.dtypes)

Duration      int64
Date         object
Pulse       float64
Maxpulse    float64
Calories    float64
dtype: object
Order ID           int64
Customer Name     object
Order Date        object
Product           object
Quantity         float64
Unit Price       float64
Total Revenue    float64
dtype: object


In [24]:
# 2. Inconsistent date formats (mine.csv)
print("\n2. Handling Inconsistent Date Formats (mine.csv):")
print("Original 'Date' dtype:", df_mine['Date'].dtype)
print("Sample of 'Date' column before conversion:\n", df_mine['Date'].head())

# removing trailing single quote

df_mine['Date'] = df_mine['Date'].astype(str).str.replace("'", "")

# Converting 'Date' column to datetime objects
df_mine['Date'] = pd.to_datetime(df_mine['Date'], errors='coerce')

#  dropping any new NaT values if the conversion failed
print("Missing 'Date' values after format coercion:\n", df_mine['Date'].isnull().sum())
if df_mine['Date'].isnull().any():
    print("Dropping rows with unparseable dates (NaT values)...")
    df_mine.dropna(subset=['Date'], inplace=True)

print("New 'Date' dtype:", df_mine['Date'].dtype)
print("Sample of 'Date' column after conversion:\n", df_mine['Date'].head())





2. Handling Inconsistent Date Formats (mine.csv):
Original 'Date' dtype: object
Sample of 'Date' column before conversion:
 0    2023/10/01'
1    2023/10/02'
2    2023/10/03'
3    2023/10/04'
4    2023/10/05'
Name: Date, dtype: object
Missing 'Date' values after format coercion:
 3
Dropping rows with unparseable dates (NaT values)...
New 'Date' dtype: datetime64[ns]
Sample of 'Date' column after conversion:
 0   2023-10-01
1   2023-10-02
2   2023-10-03
3   2023-10-04
4   2023-10-05
Name: Date, dtype: datetime64[ns]


In [25]:
# 3. Duplicate rows (mine.csv)
print("\n3. Handling Duplicate Rows in (mine.csv):")
print("Number of duplicate rows before dropping:", df_mine.duplicated().sum())
df_mine.drop_duplicates(inplace=True)
print("Number of duplicate rows after dropping:", df_mine.duplicated().sum())
print("Shape after dropping duplicates:", df_mine.shape)





3. Handling Duplicate Rows in (mine.csv):
Number of duplicate rows before dropping: 0
Number of duplicate rows after dropping: 0
Shape after dropping duplicates: (28, 5)


In [26]:
# 4. Wrong data (mine.csv)
print("\n4. Handling Wrong Data (mine.csv):")


initial_long_duration_count = df_mine[df_mine['Duration'] > 180].shape[0]
if initial_long_duration_count > 0:
    df_mine['Duration'] = np.where(df_mine['Duration'] > 180, 180, df_mine['Duration'])
    print(f"Capped {initial_long_duration_count} 'Duration' values greater than 180 to 180.")
else:
    print("No 'Duration' values greater than 180 found to cap.")

# making  all numerical columns numeric type after all operations
for col in ['Duration', 'Pulse', 'Maxpulse', 'Calories']:
    if col in df_mine.columns:
        df_mine[col] = pd.to_numeric(df_mine[col], errors='coerce')
        if df_mine[col].isnull().any():
            mean_val = df_mine[col].mean()
            df_mine[col].fillna(mean_val, inplace=True)
          

print("\n--- Cleaned mine.csv DataFrame ---")
print(df_mine)
print("\nFinal Missing values check (mine.csv):\n", df_mine.isnull().sum())
print("\nFinal Dtypes (mine.csv):\n", df_mine.dtypes)






4. Handling Wrong Data (mine.csv):
Capped 1 'Duration' values greater than 180 to 180.

--- Cleaned mine.csv DataFrame ---
    Duration       Date       Pulse    Maxpulse    Calories
0         60 2023-10-01  110.000000  130.000000  409.100000
1         60 2023-10-02  117.000000  145.000000  479.000000
2         60 2023-10-03  103.000000  135.000000  340.300000
3         45 2023-10-04  109.000000  175.000000  282.400000
4         45 2023-10-05  117.000000  150.000000  405.100000
5         60 2023-10-06  103.000000  125.000000  300.000000
6         60 2023-10-07  110.000000  135.000000  374.000000
7        180 2023-10-08  114.000000  133.000000  302.859259
8         60 2023-10-09  112.000000  126.000000  193.800000
9         30 2023-10-10  102.000000  147.000000  234.800000
10        60 2023-10-11  100.000000  129.000000  375.300000
11        60 2023-10-12  109.000000  131.000000  345.600000
12        60 2023-10-13  103.000000  136.000000  239.200000
13        60 2023-10-15  120.000000 

In [27]:
# --- Cleaning sales.csv ---
print("\n" + "="*50)
print("             CLEANING sales.csv")
print("="*50)


# 1. Missing values / Empty cells (sales.csv)
print("\n1. Handling Missing Values in (sales.csv):")
print("Missing values before handling:\n", df_sales.isnull().sum())

# 'Customer Name': Fill empty strings/NaN with 'Unknown'
if 'Customer Name' in df_sales.columns:
    df_sales['Customer Name'].fillna('Unknown', inplace=True)
    df_sales['Customer Name'] = df_sales['Customer Name'].replace('', 'Unknown') # For actual empty strings

# Filling empty cells with  mean.
if 'Quantity' in df_sales.columns and df_sales['Quantity'].isnull().any():
    median_quantity = df_sales['Quantity'].median()
    df_sales['Quantity'].fillna(median_quantity, inplace=True)
    print(f"Filled missing 'Quantity' with median: {median_quantity:.0f}")

if 'Unit Price' in df_sales.columns and df_sales['Unit Price'].isnull().any():
    median_unit_price = df_sales['Unit Price'].median()
    df_sales['Unit Price'].fillna(median_unit_price, inplace=True)
    print(f"Filled missing 'Unit Price' with median: {median_unit_price:.2f}")


if 'Total Revenue' in df_sales.columns and df_sales['Total Revenue'].isnull().any():
    median_total_revenue = df_sales['Total Revenue'].median()
    df_sales['Total Revenue'].fillna(median_total_revenue, inplace=True)
    print(f"Filled missing 'Total Revenue' with median: {median_total_revenue:.2f}")

print("Missing values after initial handling:\n", df_sales.isnull().sum())





             CLEANING sales.csv

1. Handling Missing Values in (sales.csv):
Missing values before handling:
 Order ID         0
Customer Name    0
Order Date       0
Product          0
Quantity         0
Unit Price       0
Total Revenue    0
dtype: int64
Missing values after initial handling:
 Order ID         0
Customer Name    0
Order Date       0
Product          0
Quantity         0
Unit Price       0
Total Revenue    0
dtype: int64


In [28]:
# 2. Inconsistent date formats (sales.csv)
print("\n2. Handling Inconsistent Date Formats (sales.csv):")
print("Original 'Order Date' dtype:", df_sales['Order Date'].dtype)
print("Sample of 'Order Date' column before conversion:\n", df_sales['Order Date'].head())


df_sales['Order Date'] = df_sales['Order Date'].astype(str).str.replace("'", "")

df_sales['Order Date'] = pd.to_datetime(df_sales['Order Date'], errors='coerce')

print("Missing 'Order Date' values after format coercion:\n", df_sales['Order Date'].isnull().sum())
if df_sales['Order Date'].isnull().any():
    print("Dropping rows with unparseable 'Order Date'")
    df_sales.dropna(subset=['Order Date'], inplace=True)

print("New 'Order Date' dtype:", df_sales['Order Date'].dtype)
print("Sample of 'Order Date' column after conversion:\n", df_sales['Order Date'].head())





2. Handling Inconsistent Date Formats (sales.csv):
Original 'Order Date' dtype: object
Sample of 'Order Date' column before conversion:
 0     01/01/2024
1     01/02/2024
2    2024/01/03'
3     04/01/2024
4    2024/01/05'
Name: Order Date, dtype: object
Missing 'Order Date' values after format coercion:
 2
Dropping rows with unparseable 'Order Date'
New 'Order Date' dtype: datetime64[ns]
Sample of 'Order Date' column after conversion:
 0   2024-01-01
1   2024-01-02
3   2024-04-01
5   2024-06-01
6   2024-01-01
Name: Order Date, dtype: datetime64[ns]


In [29]:
# 3. Duplicate rows (sales.csv)
print("\n3. Handling Duplicate Rows (sales.csv):")
print("Number of duplicate rows before dropping:", df_sales.duplicated().sum())
df_sales.drop_duplicates(inplace=True)
print("Number of duplicate rows after dropping:", df_sales.duplicated().sum())
print("Shape after dropping duplicates:", df_sales.shape)





3. Handling Duplicate Rows (sales.csv):
Number of duplicate rows before dropping: 1
Number of duplicate rows after dropping: 0
Shape after dropping duplicates: (5, 7)


In [30]:
# 4. Wrong data (sales.csv)
print("\n4. Handling Wrong Data (sales.csv):")
if 'Quantity' in df_sales.columns:
    initial_negative_qty_count = df_sales[df_sales['Quantity'] < 0].shape[0]
    if initial_negative_qty_count > 0:
        df_sales['Quantity'] = np.where(df_sales['Quantity'] < 0, np.nan, df_sales['Quantity'])
        print(f"Replaced {initial_negative_qty_count} negative 'Quantity' values with NaN.")
        # Re-impute quantity if new NaNs were created
        if df_sales['Quantity'].isnull().any():
            median_quantity_after_negative_fix = df_sales['Quantity'].median()
            df_sales['Quantity'].fillna(median_quantity_after_negative_fix, inplace=True)
            print(f"Filled new 'Quantity' NaNs with median: {median_quantity_after_negative_fix:.0f}")
    else:
        print("No negative 'Quantity' values found.")

# to  numeric
for col in ['Unit Price', 'Total Revenue']:
    if col in df_sales.columns:
        df_sales[col] = pd.to_numeric(df_sales[col], errors='coerce')
        if df_sales[col].isnull().any():
            median_val = df_sales[col].median()
            df_sales[col].fillna(median_val, inplace=True)
            print(f"Ensured '{col}' is numeric and filled any new NaNs with median: {median_val:.2f}")

# Recalculating 'Total Revenue' to ensure consistency, because Quantity/Unit Price were changed

if 'Quantity' in df_sales.columns and 'Unit Price' in df_sales.columns and 'Total Revenue' in df_sales.columns:
    df_sales['Calculated Revenue'] = df_sales['Quantity'] * df_sales['Unit Price']
   
    print("Recalculated 'Total Revenue' based on 'Quantity' * 'Unit Price'.")






4. Handling Wrong Data (sales.csv):
Replaced 1 negative 'Quantity' values with NaN.
Filled new 'Quantity' NaNs with median: 4
Recalculated 'Total Revenue' based on 'Quantity' * 'Unit Price'.


C:\Users\MAKOS\AppData\Local\Temp\ipykernel_17108\886480360.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_sales['Quantity'].fillna(median_quantity_after_negative_fix, inplace=True)


In [31]:
# 5. Unnecessary columns that are not relevant to the analysis (sales.csv)
print("\n5. Removing Unnecessary Columns (sales.csv):")
print("Columns before dropping:", df_sales.columns.tolist())

# Assuming 'Order ID' might be unique enough and not needed for aggregate analysis

columns_to_drop_sales = [] # No obvious unnecessary columns in your sales.csv sample for now


existing_columns_to_drop_sales = [col for col in columns_to_drop_sales if col in df_sales.columns]

if existing_columns_to_drop_sales:
    df_sales.drop(columns=existing_columns_to_drop_sales, inplace=True)
    print(f"Dropped columns: {existing_columns_to_drop_sales}")
else:
    print("No specified unnecessary columns found to drop in sales.csv.")

print("Columns after dropping:", df_sales.columns.tolist())

print("\n--- Cleaned sales.csv DataFrame ---")
print(df_sales)
print("\nFinal Missing values check (sales.csv):\n", df_sales.isnull().sum())
print("\nFinal Dtypes (sales.csv):\n", df_sales.dtypes)



5. Removing Unnecessary Columns (sales.csv):
Columns before dropping: ['Order ID', 'Customer Name', 'Order Date', 'Product', 'Quantity', 'Unit Price', 'Total Revenue', 'Calculated Revenue']
No specified unnecessary columns found to drop in sales.csv.
Columns after dropping: ['Order ID', 'Customer Name', 'Order Date', 'Product', 'Quantity', 'Unit Price', 'Total Revenue', 'Calculated Revenue']

--- Cleaned sales.csv DataFrame ---
   Order ID  Customer Name Order Date   Product  Quantity  Unit Price  \
0      1001       John Doe 2024-01-01  Widget A      10.0   25.000000   
1      1002     Jane Smith 2024-01-02  Widget B       5.0   40.000000   
3      1004  Alice Johnson 2024-04-01  Widget C       3.0   35.714286   
5      1006       John Doe 2024-06-01  Widget A       4.0   25.000000   
7      1007     Jane Smith 2024-07-01  Widget C       4.5   70.000000   

   Total Revenue  Calculated Revenue  
0          250.0          250.000000  
1          200.0          200.000000  
3          